In [1]:
import sagemaker
import boto3

from sagemaker.pytorch import PyTorch
# from sagemaker.huggingface import HuggingFace
from sagemaker.inputs import TrainingInput
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::940119374655:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole
sagemaker bucket: sagemaker-us-east-1-940119374655
sagemaker session region: us-east-1


In [3]:
student_id = "mistralai/Mistral-7B-v0.1"
teacher_id = "mistralai/Mixtral-8x7B-v0.1"
dataset_id = "imdb"
s3_prefix_dataset = "knowledge_distill_mistral_classification"

In [4]:
dataset_path = f's3://{sess.default_bucket()}/{s3_prefix_dataset}'
train_dataset_path = dataset_path + '/dataset/train'
test_dataset_path = dataset_path + '/dataset/test'

In [5]:
script = '18.py'
train_name = script.replace('_', '-')

In [6]:
job_name = f'{train_name[:-3]}-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
print(job_name)

18-2024-03-31-17-51-56


In [7]:
hyperparameters={
        'teacher_model_id': teacher_id,
        'student_model_id': student_id,
        'num_epochs': 1,
        'lr': 6e-5,
        'fp16': True,
        'temperature': 4.0,
        'alpha': 0.5,
        'save_strategy': 'epoch',
        'evaluation_strategy': 'epoch',
}

In [8]:
distribution = {
    "torch_distributed":{
        "enabled": True
    },
    "smdistributed": {
        "modelparallel": {
            "enabled": True,
            "parameters": {
                "hybrid_shard_degree": 0,
            }
        }
    }
}

In [9]:
pytorch_estimator = PyTorch(
    entry_point          = script,
    source_dir           = 'scripts',
    instance_type        = 'ml.p4d.24xlarge',
    instance_count       = 2,
    base_job_name        = job_name,
    role                 = role,
    framework_version    = '2.2.0',
    py_version           = 'py310',
    hyperparameters      = hyperparameters,
    distribution         = distribution
)

In [10]:
# mpi_options = {
#     "enabled" : True,
#     "processes_per_host" : 8,
# }
# smp_options = {
#     "enabled":True,
#     "parameters": {
#         "microbatches": 4,
#         "placement_strategy": "spread",
#         "pipeline": "interleaved",
#         "optimize": "speed",
#         "partitions": 4,
#         "ddp": True,
#     }
# }

# distribution={
#     "smdistributed": {"modelparallel": smp_options},
#     "mpi": mpi_options
# }

In [11]:
# hf_estimator = HuggingFace(
#     entry_point          = script,
#     source_dir           = 'scripts',
#     instance_type        = 'ml.p4d.24xlarge',
#     instance_count       = 2,
#     base_job_name        = job_name,
#     role                 = role,
#     transformers_version = '4.36',
#     pytorch_version      = '2.1',
#     py_version           = 'py310',
#     hyperparameters      = hyperparameters,
#     distribution         = distribution
# )

In [12]:
pytorch_estimator.fit(inputs={
    'train': TrainingInput(
        s3_data=train_dataset_path,
        input_mode='FastFile'),
    'test': TrainingInput(
        s3_data=test_dataset_path,
        input_mode='FastFile'),
})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: 18-2024-03-31-17-51-56-2024-03-31-17-51-56-423


2024-03-31 17:51:57 Starting - Starting the training job
2024-03-31 17:51:57 Pending - Training job waiting for capacity......
2024-03-31 17:52:43 Pending - Preparing the instances for training...........................
2024-03-31 17:57:05 Downloading - Downloading input data...
2024-03-31 17:57:45 Downloading - Downloading the training image............
2024-03-31 17:59:50 Training - Training image download completed. Training in progress......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-03-31 18:00:40,098 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-03-31 18:00:40,200 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-03-31 18:00:40,209 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-03-31 18:00:40,211 sagemaker_pytorch_container.training INFO     Invoking TorchDistributed..

UnexpectedStatusException: Error for Training job 18-2024-03-31-17-51-56-2024-03-31-17-51-56-423: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "raise ValueError(f"TP policy does not exist for {type(module)}, can't transform.")
 transformed_model = state.tp_registry.distribute(model, tp_config=config)ValueError
 TP policy does not exist for <class 'transformers.models.mixtral.modeling_mixtral.MixtralForSequenceClassification'>, can't transform.  File "/opt/conda/lib/python3.10/site-packages/torch/sagemaker/tensor_parallel/tp_registry.py", line 48, in distribute
 main()
 File "/opt/ml/code/18.py", line 191, in main
 training_function(args)
 File "/opt/ml/code/18.py", line 154, in training_function
 Traceback (most recent call last)
 ValueError    training_function(args)
 TP policy does not exist for <class 'transformers.models.mixtral.modeling_mixtral.MixtralForSequenceClassification'>, can't transform.
 File "/opt/ml/code/18.py", line 195, in <module>


In [ ]:
pip install -U sagemaker